In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [2]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [3]:
df = pd.read_sql(
    """
    select date, ticker, acc, agr, beta, bm, ep, gma, idiovol,
    lev, mom12m, mom1m, mve, operprof, roeq, ret
    from data
    order by date, ticker
    """,
    conn
  )
df = df.dropna()
df = df.set_index(["date", "ticker"])

In [4]:
transform = QuantileTransformer(output_distribution="normal")
for col in df.columns:
    df[col] = df.groupby("date", group_keys=False)[col].transform(transform)



AssertionError: 

In [ ]:
transform1 = QuantileTransformer(
    output_distribution="normal"
)

transform2 = QuantileTransformer(
    output_distribution="normal"
)

poly = PolynomialFeatures(degree=2)

model = RandomForestRegressor(
   random_state=0
)

pipe = make_pipeline(
  poly,
  model
)

In [ ]:
later = df.index.get_level_values("date")>="2010-01"

train = df[~later]
test = df[later]

In [ ]:
predictors = [
  "acc", "agr", "beta", "bm", "ep", "gma", "idiovol",
  "lev", "mom12m", "mom1m", "mve", "operprof", "roeq"
]

Xtrain = train[predictors]
ytrain = train["ret"]

_ = pipe.fit(Xtrain, ytrain)

In [ ]:
Xtest = test[predictors]

ypredict = model.predict(Xtest)
ypredict = pd.Series(ypredict, index=test.index)

In [1]:
from sqlalchemy import create_engine
import pandas as pd
from modules.backtest import cumplot, mvplot, regress
import datapane as dp
import plotly.graph_objects as go


In [ ]:
from joblib import load 
forest = load("forest2.joblib")

from sqlalchemy import create_engine
import pymssql
import pandas as pd

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m, siccd
    from data
    where date='2022-01'
    """, 
    conn
)
data = data.dropna()

X = data[["roeq", "mom12m"]]
data["predict"] = forest.predict(X)

In [ ]:
data = data.sort_values(
  by="predict",
  ascending=False
)
data = data.reset_index(drop=True)

In [2]:
with create_engine("sqlite:///files/mydata.db").connect() as conn:
    rets = pd.read_sql("select * from model1", conn)
    rets["market"] = pd.read_sql("select market from market", conn)
    rets["rf"] = pd.read_sql("select rf from market", conn)

In [3]:
text = """
    The features are roeq and mom12m.  The feature pipeline is (QuantileTransformer, 
    PolynomialFeatures, QuantileTransformer).  The target is the return minus the
    market return and ias transformed with QuantileTransformer.  Random forests with
    max_depth=4 are trained at five-year intervals and used to predict monthly.  The best
    and worst returns are the equally weighted returns of the top 100 and bottom 100 stocks
    each month, respectively.
"""

figa = cumplot(rets)
figb = mvplot(rets)
tbla = regress(rets, "best")
tblb = regress(rets, "worst")

In [11]:
pg = dp.Page(
    title="Random Forest",
    blocks = [
        "## Random Forest Model",
        "Kerry Back, November 28, 2022",
        text,
        dp.Select(
            dp.Plot(
                figa, 
                caption="Cumulative Returns",
                label="Cumulative returns"
            ),
            dp.DataTable(
                tbla, 
                caption="""
                    Regression of Best-RF on Mkt-RF.  The alpha estimate is
                    in percent per year.
                """,
                label="Alpha and beta of best portfolio"
            ),
            dp.DataTable(
                tblb, 
                caption="""
                    Regression of Worst-RF on Mkt-RF.  The alpha estimate is
                    in percent per year.
                """,
                label="Alpha and beta of worst portfolio"
            ),
            dp.Plot(
                figb, 
                caption="Mean-Variance Analysis",
                label="Mean-variance analysis"
            ),
        ),
    ]
)

app = dp.App(
    pg,
)
app.save("files/dashboard.html")


c:\Users\kerry\repos\2022-638-binder\venv\lib\site-packages\datapane\common\df_processor.py:28: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



App saved to ./files/dashboard.html

If you want to create an html file with separate pages for different models, you can create each page as in the above and then use something like

    app = dp.App(
        pg1, 
        pg2,
        pg3
    )



datapane.com will host your html file for free.  Just create an account, get your login token and add the following two lines to your notebook.  You will then get a link that you can share.

    !datapane login --token=whateverYourTokenIs
    app.upload(name="WhateverYouWantToCallIt")

In [5]:
# Example
# 
# !datapane login --token=
# app.upload(name="Investment Strategy Backtests")

In [ ]:
ff49 = pd.read_excel("ff49.xlsx", index_col=0)
def industry(sic):
  try:
    return ff49[(ff49.lower<=sic)&(sic<=ff49.upper)].index[0]
  except:
    return "Almost Nothing"
top["industry"] = top.siccd.map(industry)
counts = top.groupby("industry").ticker.count()
counts = counts.rename(index={"Almost Nothing": "Other"})
counts = counts.reset_index()
counts.columns = ["industry", "count"]

import plotly.express as px
fig = px.pie(counts, values="count", names="industry")

fig.update_layout(
    template="plotly_dark",
    font_size=20,
    width=1000,
    height=800,
)

fig.show()
